In [56]:
video_path = 'GOPR1077_tr.mp4'
model_path = 'best.pt' # path to .pt file from YOLOv10 notebook

# dimensions of photos the model was trained on
IMAGE_WIDTH = 416
IMAGE_HEIGHT = 416

In [57]:
!pip install -q av pillow

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [58]:
from ultralytics import YOLOv10
from PIL import Image
import numpy as np
import supervision as sv
import cv2
import av, os
from PIL import Image

In [59]:

# im_pil = Image.open("oyster_test.jpg")
# image = np.array(im_pil)https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov10-object-detection-on-custom-dataset.ipynb
# image_2 = np.array(Image.open('problem_photo_small.jpg'))
# print(im_pil.size)
# ann_img = run_pred(image_2)
# sv.plot_image(ann_img)

In [ ]:
def annotate_image(input_img, label_annotator, bounding_box_annotator, model_pred, verbose = False):
    cont_img = np.ascontiguousarray(input_img, dtype=np.uint8)
    results = model_pred(cont_img, conf=0.10, verbose = verbose)[0]
    detections = sv.Detections.from_ultralytics(results)
    annotated_image = bounding_box_annotator.annotate(
        scene=input_img, detections=detections)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections)
    return(annotated_image)

def annotate_video(in_path, out_path = '.', model_file = 'best.pt', im_width = IMAGE_WIDTH, im_height = IMAGE_HEIGHT):
    bba = sv.BoundingBoxAnnotator()
    la = sv.LabelAnnotator()
    model = YOLOv10(model_file)

    container = av.open(in_path)
    stream_vid = container.streams.video[0]
    fname = in_path.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    out_path = f'{out_path}/{fname[:per_index]}_annotated.mp4'
    outp = av.open(out_path, 'w')
    codec_name = stream_vid.codec_context.name
    fps = stream_vid.codec_context.rate
    output_stream = outp.add_stream(codec_name, str(fps))
    output_stream.width = im_width
    output_stream.height = im_height
    output_stream.pix_fmt = stream_vid.codec_context.pix_fmt
    for index, frame in enumerate(container.decode(stream_vid)):
        pil_img = frame.to_image()
        np_img = np.array(pil_img)
        np_img_resize = cv2.resize(np_img, (im_width, im_height))
        np_rot = np_img_resize[:, :, ::-1]
        small_pil_img = Image.fromarray(np_rot)
        np_image_2 = np.array(small_pil_img)
        an_mg = annotate_image(np_image_2, label_annotator = la, bounding_box_annotator=bba, model_pred = model, verbose = False)
        frame_out = av.VideoFrame.from_ndarray(an_mg, format='bgr24')
        pkt = output_stream.encode(frame_out)
        outp.mux(pkt)
    container.close()
    outp.close()

    print(f'Annotated video has been saved as {out_path}')
    return True

Annotated video has been saved as GOPR1077_tr_annotated.mp4


In [85]:
# without any modification == 41.5
# with modification == 10

10
